In [1]:
import pandas as pd
import re

In [2]:
#### initial number of the floor is -1
floor = []
### read the cleaned rental csv file
new_property_df = pd.read_csv('../data/curated/cleaned_property_with_rent.csv')
for i in range(len(new_property_df["address"])):
    floor.append('-1')
### create the floor feature
new_property_df["floor"] = floor
new_property_df = new_property_df.copy()
###check the new csv
new_property_df.head()

,address,rent,features,type,furnitured,pool,gym,latitude,longitude,postcode,num_bed,num_bath,num_park,floor
0,1414/218-228 A'Beckett Street Melbourne VIC 3000,$400 Per Week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810283,144.956669,3000,1,1,0,-1
1,11a/131 Lonsdale Sreet Melbourne VIC 3000,$350 per week,1 Bed1 Bath− Parking,Studio,Yes,No,No,-37.810779,144.968551,3000,1,1,0,-1
2,911/408 Lonsdale Street Melbourne VIC 3000,$330 per week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812598,144.960401,3000,1,1,0,-1
3,918/422 Collins St Melbourne VIC 3000,$600 Per week fully furnished,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.817097,144.960149,3000,2,1,0,-1
4,602/118 Franklin Street Melbourne VIC 3000,$330,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.808205,144.958903,3000,1,1,0,-1


In [3]:
for i in range(len(new_property_df["address"])):
    # change the building that only have one floor only (such as Villa)
    if (new_property_df["type"][i] == "House") or (new_property_df["type"][i] == "Villa") or (new_property_df["type"][i] == "New House & Land") or (new_property_df["type"][i] == "Semi-Detached") or (new_property_df["type"][i] == "Vacant land"):
        new_property_df.loc[[i],['floor']]  = 1
    elif (new_property_df["type"][i] == "Rural") or (new_property_df["type"][i] == "Farm"):
        new_property_df.loc[[i],['floor']]  = 1
    else:
        location = new_property_df["address"][i].split(" ")[0]
        if "/" in location:  
            floor_str = location.split("/")
            ### create temporary variables to store the floor information
            f=''
            fff=''
            ### process the floor address is not digital.
            if floor_str[0].isdigit() == False: 
                ###special case
                ### ground floor
                if ("G" in floor_str[0]) or ("Ground" in floor_str[0]):
                    new_property_df.loc[[i],['floor']]  = 1
                if ("LG" in floor_str[0]):
                    new_property_df.loc[[i],['floor']]  = 1
                ## switch the format of the floor address from various form to uniform digitals.
                if ("Level" in floor_str[0]) or ("L" in floor_str[0]):  
                    if len(re.findall('[0-9]', floor_str[0])) != 0:
                        new_property_df.loc[[i],['floor']] = int(f.join(re.findall("[0-9]", floor_str[0])))
                elif ("&" in floor_str[0]):
                    new_property_df.loc[[i],['floor']]  = floor_str[0][0]
                elif ("." in floor_str[0]):
                    if ("AP" in floor_str[0]):
                        new_property_df.loc[[i],['floor']]  = floor_str[0].split(".")[0][-1] 
                    else:
                        if len(re.findall('[0-9]', floor_str[0].split('.')[0])) != 0:
                            new_property_df.loc[[i],['floor']]  = int(fff.join(re.findall('[0-9]', floor_str[0].split('.')[0])))
                else:
                    ####### Determine the floor number when there is a letter on the end of the floor address such as '1014B'
                    if re.search('(\\d+)[A-Za-z]',new_property_df["address"][i]) != None:
                       tmp_floor = re.search('(\\d+)[A-Za-z]',new_property_df["address"][i]).group()[:-1]
                       if len(tmp_floor) == 1 or len(tmp_floor) == 2:
                           new_property_df.loc[[i],['floor']]  = tmp_floor
                       elif len(tmp_floor) == 3 and int(tmp_floor[1]) > 1:
                           new_property_df.loc[[i],['floor']]  = tmp_floor[:2]
                       elif len(tmp_floor) == 3 and int(tmp_floor[1]) <= 1:
                           new_property_df.loc[[i],['floor']]  = tmp_floor[0]
                       elif len(tmp_floor) == 4:
                           new_property_df.loc[[i],['floor']]  = tmp_floor[:2]                   
            ### determine which number is the floor. for example: 319 is 3th floor and 3109 is the 31th floor.
            else:
                if len(floor_str[0]) == 1 or len(floor_str[0]) == 2:
                    new_property_df.loc[[i],['floor']]  = floor_str[0]
                elif len(floor_str[0]) == 3 and int(floor_str[0][1]) > 1:
                    new_property_df.loc[[i],['floor']]  = floor_str[0][:2]
                elif len(floor_str[0]) == 3 and int(floor_str[0][1]) <= 1:
                    new_property_df.loc[[i],['floor']]  = floor_str[0][0]
                elif len(floor_str[0]) == 4:
                    new_property_df.loc[[i],['floor']]  = floor_str[0][:2]
        ### process the address without '/'
        else: 
            ### check the floors that have "," on the tail.       
            if ',' in new_property_df["address"][i]:
               floor_str111 = new_property_df["address"][i].split(",")[0]
               if len(floor_str111.split(' '))>1:
                  FL = floor_str111.split(' ')[0]
                  FF = floor_str111.split(' ')[1]
                  if ("Level" in FL) or ("Lvl" in FL):  
                     new_property_df.loc[[i],['floor']] = FF
            ####### Determine the floor number when there is a letter on the end of the floor address such as '1014B'
            if re.search('(\\d+)[A-Za-z]',new_property_df["address"][i]) != None:
                tmp_floor = re.search('(\\d+)[A-Za-z]',new_property_df["address"][i]).group()[:-1]
                if len(tmp_floor) == 1 or len(tmp_floor) == 2:
                     new_property_df.loc[[i],['floor']]  = tmp_floor
                elif len(tmp_floor) == 3 and int(tmp_floor[1]) > 1:
                     new_property_df.loc[[i],['floor']]  = tmp_floor[:2]
                elif len(tmp_floor) == 3 and int(tmp_floor[1]) <= 1:
                     new_property_df.loc[[i],['floor']]  = tmp_floor[0]
                elif len(tmp_floor) == 4:
                     new_property_df.loc[[i],['floor']]  = tmp_floor[:2]  

In [4]:
count=0
### test the results that has no floor information
for i in range(len(new_property_df["address"])):
    location = new_property_df["address"][i].split('/')[0]
    if new_property_df["floor"][i] == '-1':
        
        print(new_property_df["address"][i])
        print(new_property_df["floor"][i])
        count+=1
print("Count of the address without floor information: ", count)
### check the shape of the cleaned dataframe
new_property_df.shape

Unit 1713/200 Spencer St Melbourne VIC 3000
-1
394 Collins Street Melbourne VIC 3000
-1
FLAT 212/500 FLINDERS STREET Melbourne VIC 3000
-1
CARPARK/442 Elizabeth Street Melbourne VIC 3000
-1
593 Elizabeth Street Melbourne VIC 3000
-1
UNIT 2707/35 SPRING STREET Melbourne VIC 3000
-1
285 La Trobe Street Melbourne VIC 3000
-1
Melbourne VIC 3000
-1
325 Collins Street Melbourne VIC 3000
-1
555 Flinders St Melbourne VIC 3000
-1
9-23 MacKenzie Street Melbourne VIC 3000
-1
311 EXHIBITION ST Melbourne VIC 3000
-1
unit 604/77 Queens Road Melbourne VIC 3000
-1
81 Abeckett Street Melbourne VIC 3000
-1
327 La Trobe Street Melbourne VIC 3000
-1
unit 1701/141-149 La Trobe Street Melbourne VIC 3000
-1
260 Spencer St Melbourne VIC 3000
-1
260 Spencer St Melbourne VIC 3000
-1
Elizabeth street Melbourne VIC 3000
-1
327 La Trobe Street Melbourne VIC 3000
-1
350 William Street Melbourne VIC 3000
-1
250 Elizabeth Street Melbourne VIC 3000
-1
Melbourne VIC 3000
-1
Unit 101/170 Albert St East Melbourne VIC 300

(14819, 14)

In [7]:
### check the cleaned dataframe
new_property_df.head(20)

,address,rent,features,type,furnitured,pool,gym,latitude,longitude,postcode,num_bed,num_bath,num_park,floor
0,1414/218-228 A'Beckett Street Melbourne VIC 3000,$400 Per Week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810283,144.956669,3000,1,1,0,14
1,11a/131 Lonsdale Sreet Melbourne VIC 3000,$350 per week,1 Bed1 Bath− Parking,Studio,Yes,No,No,-37.810779,144.968551,3000,1,1,0,11
2,911/408 Lonsdale Street Melbourne VIC 3000,$330 per week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812598,144.960401,3000,1,1,0,9
3,918/422 Collins St Melbourne VIC 3000,$600 Per week fully furnished,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.817097,144.960149,3000,2,1,0,9
4,602/118 Franklin Street Melbourne VIC 3000,$330,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.808205,144.958903,3000,1,1,0,6
5,1112/333-351 Exhibition Street Melbourne VIC 3000,$600 per week,2 Beds2 Baths− Parking,Apartment / Unit / Flat,Yes,No,No,-37.807896,144.968287,3000,2,2,0,11
6,3002/288 Spencer St Melbourne VIC 3000,$510,2 Beds1 Bath1 Parking,Apartment / Unit / Flat,No,Yes,Yes,-37.813775,144.952095,3000,2,1,1,30
7,4/180 Little Collins Street Melbourne VIC 3000,$500 per week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,Yes,No,No,-37.813860,144.967907,3000,1,1,0,4
8,1605/565 Flinders Street Melbourne VIC 3000,$500 per week,2 Beds2 Baths1 Parking,Apartment / Unit / Flat,No,No,No,-37.821059,144.955907,3000,2,2,1,16
9,612/408 Lonsdale Street Melbourne VIC 3000,$350,1 Bed1 Bath− Parking,Apartment / Unit / Flat,Yes,No,No,-37.812598,144.960401,3000,1,1,0,6


In [8]:
### save the new dataframe to csv
new_property_df.to_csv('../data/curated/cleaned_property_with_floor.csv')